In [1]:
import pandas as pd

file_path = "/data/datasets/rishi/symptom_classification/data/cvat_labels_12-1.csv"
df = pd.read_csv(file_path)

print(df.head())

  video_name   frame  Redness  Pus  Brodsky Tonsil Size  Mallampati
0       0347  211081      2.0  0.0                  1.0         1.0
1       0348  211365      1.0  0.0                  0.0         1.0
2       0353  213126      1.0  0.0                  3.0         2.0
3       0355  213761      1.0  0.0                  4.0         1.0
4       0356  214312      2.0  2.0                  4.0         2.0


In [2]:
print("Filled values for Redness, Pus, Tonsil Size, Mallampati:")
print(df[df['Redness'].notnull()].shape[0], df[df['Pus'].notnull()].shape[0], df[df['Brodsky Tonsil Size'].notnull()].shape[0], df[df['Mallampati'].notnull()].shape[0], "\n")

# distribution of redness values:
print("redness value counts:")
print(df[df['Redness'].notnull()]['Redness'].value_counts(), "\n")

# distribution of pus values:
print("pus value counts:")
print(df[df['Pus'].notnull()]['Pus'].value_counts(), "\n")

# distribution of tonsil size values:
print("tonsil size value counts:")
print(df[df['Brodsky Tonsil Size'].notnull()]['Brodsky Tonsil Size'].value_counts(), "\n")

# distribution of mallampati values:
print("mallampati value counts:")
print(df[df['Mallampati'].notnull()]['Mallampati'].value_counts(), "\n")

Filled values for Redness, Pus, Tonsil Size, Mallampati:
135 135 135 136 

redness value counts:
2.0     60
1.0     57
0.0     16
99.0     2
Name: Redness, dtype: int64 

pus value counts:
0.0     90
1.0     23
2.0     13
99.0     9
Name: Pus, dtype: int64 

tonsil size value counts:
3.0     38
2.0     33
4.0     33
1.0     22
99.0     5
0.0      4
Name: Brodsky Tonsil Size, dtype: int64 

mallampati value counts:
1.0    85
2.0    44
3.0     7
Name: Mallampati, dtype: int64 



In [32]:

'''
file_path = "/data/datasets/rishi/symptom_classification/data/training_review_tc_3.14.23.xlsx"
df = pd.read_excel(file_path)
SYMPTOM = 'Pus'

# Remove missing entries (for now, just tonsil size)
df = df[df[SYMPTOM] != 99]

# Extract patient IDs and frame names
df['Patient_ID'] = df['Frame Name'].str.extract(r'^(.*?)-frame_')
df['Frame'] = df['Frame Name'].str.extract(r'(frame_\d+)')
# new_df = pd.read_csv('path_to_your_new_csv.csv')
'''

df = pd.read_csv('/data/datasets/rishi/symptom_classification/data/combined_test_data_Pus_1.csv')

cvat_path = "/data/datasets/rishi/symptom_classification/data/combined_train_data_Pus_1.csv"
cvat_df = pd.read_csv(cvat_path)

# Convert 'Patient ID' and 'video_name' columns to sets
patient_id_set = set(df['Patient_ID'])
video_name_set = set(cvat_df['Patient_ID'])

# Find the intersection of the two sets
overlap = patient_id_set.intersection(video_name_set)

# Check if there is any overlap
if overlap:
    print("There is overlap. The overlapping elements are:", overlap)
else:
    print("There is no overlap between the two datasets.")

unique_to_df = patient_id_set.difference(video_name_set)

# Find IDs unique to cvat_df (video_name_set)
unique_to_cvat_df = video_name_set.difference(patient_id_set)

# Check and print the unique IDs
if unique_to_df:
    print("IDs in df but not in cvat_df:", unique_to_df)
else:
    print("There are no unique IDs in df compared to cvat_df.")

if unique_to_cvat_df:
    print("IDs in cvat_df but not in df:", unique_to_cvat_df)
else:
    print("There are no unique IDs in cvat_df compared to df.")

There is no overlap between the two datasets.
IDs in df but not in cvat_df: {'0094', '0176', '0068', '0408', '0433', '0604', '0639-1', '0393', '0461', '0514', '0355', '0123', '0372', '0366', '0165', '0533', '0347', '0339', '0551', '0069', '0541', '0008', '0276', '0475-2', '0468', '0640', '5594-7', '0497', '0625', '0489-2', '0455', '0465-1', '0576-2', '0277', '0414-2', '0288', '0595', '0189', '0454', '0233', '0102', '0443', '0426', '5586-22', '0464', '0495', '5586-39', '0325', '0572'}
IDs in cvat_df but not in df: {'0635', '0133', '0405', '0297', '0585', '5586-16', '0542', '0057-1', '0114', '0399', '0186', '0249', '0496', '0002', '0164', '0404', '0609', '0502', '0550', '0326', '5586-5', '0006', '0501', '0570', '5590-7', '0419', '0370', '0003', '0151', '0227', '0456', '0356', '0592', '0598', '5586-49', '0532', '5588-5', '0247', '0157', '0155', '0618', '0476', '0573', '0015', '0486', '0593-2', '0437', '0364', '6129-6', '0373', '0035', '0629', '0601', '0348', '0020-2', '0331', '0257', '558

In [30]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = "/data/datasets/rishi/symptom_classification/data/training_review_tc_3.14.23.xlsx"
df = pd.read_excel(file_path)

SYMPTOM = 'Pus'
SEED = 1

# Remove missing entries (for now, just tonsil size)
df = df[df[SYMPTOM] != 99]

# Extract patient IDs and frame names
df['Patient_ID'] = df['Frame Name'].str.extract(r'^(.*?)-frame_')
df['Frame'] = df['Frame Name'].str.extract(r'(frame_\d+)')

print("Number of unique patients:", len(df['Patient_ID'].unique()))

# Compute the image paths based on patient IDs and frame names
df['Image_Path'] = '/data/datasets/rishi/cropped_frames/' + df['Patient_ID'] + '/' + df['Frame'] + '.jpg'
df = df[df['Image_Path'].apply(os.path.exists)]

columns_to_keep = ['Patient_ID', 'Image_Path', 'Mallampati', 'In Focus', 'Use to Train', 'Redness', 'Pus', 'Tonsil Size', 'Strep Positive']
df = df[columns_to_keep]

# Load new data
new_data_path = "/data/datasets/rishi/symptom_classification/data/cvat_labels_12-1.csv"  # Update this path
new_df = pd.read_csv(new_data_path)

# Rename columns in the new dataframe to match the existing dataframe
new_df = new_df.rename(columns={'video_name': 'Patient_ID', 'Brodsky Tonsil Size': 'Tonsil Size'})

# Compute image paths for the new dataframe
new_df['Frame'] = 'frame_000' + new_df['frame'].astype(str)
new_df['Image_Path'] = '/data/datasets/rishi/cropped_frames/' + new_df['Patient_ID'] + '/' + new_df['Frame'] + '.jpg'
new_df.drop(columns=['Frame'], inplace=True)

# Combine the new dataframe with the existing dataframe
combined_df = pd.concat([df, new_df], ignore_index=True)

print("before dropping duplicates:", combined_df.shape)

combined_df = combined_df.drop_duplicates(subset=['Image_Path'])

print("after dropping duplicates:", combined_df.shape)

# Filter out rows where Pus is 99 or null
combined_df = combined_df[combined_df['Pus'].notnull() & (combined_df['Pus'] != 99)]

#map pus values of 2 to 1
def map_pus_values(val):
    if val in [2.0]:
        return 1.0
    return val

# Apply the function to the Pus column
combined_df['Pus'] = combined_df['Pus'].apply(map_pus_values)

#drop frame, in focus, strep positive, and use to train columns
combined_df = combined_df.drop(columns=['frame', 'In Focus', 'Strep Positive', 'Use to Train'])

print("after dropping pus 99 and null values:")
print(combined_df.shape)

# Filter out rows where images don't exist
combined_df = combined_df[combined_df['Image_Path'].apply(os.path.exists)]

# Split data into train and test based on patient IDs
patient_ids = combined_df['Patient_ID'].unique()
train_patient_ids, test_patient_ids = train_test_split(patient_ids, test_size=0.3, random_state=SEED)

train_df = combined_df[combined_df['Patient_ID'].isin(train_patient_ids)]
test_df = combined_df[combined_df['Patient_ID'].isin(test_patient_ids)]

train_csv_path = f'/data/datasets/rishi/symptom_classification/data/combined_train_data_{SYMPTOM}_{SEED}.csv'
test_csv_path = f'/data/datasets/rishi/symptom_classification/data/combined_test_data_{SYMPTOM}_{SEED}.csv'

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print("after dropping pus 99 and null values:")
print(combined_df.shape)

df = combined_df

print("Filled values for Redness, Pus, Tonsil Size, Mallampati:")
print(df[df['Redness'].notnull()].shape[0], df[df['Pus'].notnull()].shape[0], df[df['Tonsil Size'].notnull()].shape[0], df[df['Mallampati'].notnull()].shape[0], "\n")

# distribution of redness values:
print("redness value counts:")
print(df[df['Redness'].notnull()]['Redness'].value_counts(), "\n")

# distribution of pus values:
print("pus value counts:")
print(df[df['Pus'].notnull()]['Pus'].value_counts(), "\n")

# distribution of tonsil size values:
print("tonsil size value counts:")
print(df[df['Tonsil Size'].notnull()]['Tonsil Size'].value_counts(), "\n")

# distribution of mallampati values:
print("mallampati value counts:")
print(df[df['Mallampati'].notnull()]['Mallampati'].value_counts(), "\n")


Number of unique patients: 168
before dropping duplicates: (555, 10)
after dropping duplicates: (518, 10)
after dropping pus 99 and null values:
(501, 6)
after dropping pus 99 and null values:
(463, 6)
Filled values for Redness, Pus, Tonsil Size, Mallampati:
463 463 463 461 

redness value counts:
1.0     383
0.0      61
2.0      18
99.0      1
Name: Redness, dtype: int64 

pus value counts:
0.0    265
1.0    198
Name: Pus, dtype: int64 

tonsil size value counts:
1.0     141
0.0      85
3.0      76
2.0      70
4.0      56
99.0     35
Name: Tonsil Size, dtype: int64 

mallampati value counts:
1.0    240
2.0    180
3.0     35
4.0      6
Name: Mallampati, dtype: int64 



In [22]:

# Assuming combined_df is your merged DataFrame

# Define the label columns to check
label_columns = ['Mallampati', 'Redness', 'Pus', 'Tonsil Size']

df = df[df['Pus'] != 99]

# Function to check if all rows in each group have the same values for label columns
def check_label_consistency(group):
    # Compare each label column to the first row of the group
    for col in label_columns:
        if not (group[col] == group.iloc[0][col]).all():
            return False
    return True

# Apply the function to each group
consistency_check = df.groupby('Patient_ID').apply(check_label_consistency)

# Find patient IDs where labels are not consistent
inconsistent_patients = consistency_check[consistency_check == False].index.tolist()

# Output the result
if inconsistent_patients:
    print("Inconsistent labels found for Patient IDs:", inconsistent_patients)
else:
    print("All labels are consistent across rows with the same Patient_ID.")


Inconsistent labels found for Patient IDs: ['0004', '0068', '0069', '0078', '0097', '0102', '0114', '0115', '0136-2', '0159', '0162', '0164', '0165', '0168', '0227', '0276', '0277', '0282', '0325', '0348', '0355', '0366', '0370', '0372', '0376', '0392', '0412', '0414-2', '0424', '0426', '0433', '0441', '0443', '0446', '0447', '0454', '0455', '0464', '0465-1', '0473', '0476', '0486', '0495', '0496', '0499', '0501', '0502', '0532', '0533', '0542', '0569-1', '0572', '0573', '0585', '0591', '0593-2', '0597', '0610', '0640', '5586-22']
